<a href="https://colab.research.google.com/github/lucapericlp/lora-latencies/blob/master/LoRA_inference_latencies_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate evaluate datasets peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00


In [2]:
import transformers
import accelerate
import peft

print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")

Transformers version: 4.35.2
Accelerate version: 0.24.1
PEFT version: 0.6.2


In [3]:
model_checkpoint = "google/vit-base-patch16-224-in21k"
from datasets import load_dataset

dataset = load_dataset("food101", split="train[:5000]")
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

In [4]:
from PIL import Image
import requests

url = "https://huggingface.co/datasets/sayakpaul/sample-datasets/resolve/main/beignets.jpeg"
image = Image.open(requests.get(url, stream=True).raw)

In [5]:
from transformers import AutoImageProcessor

repo_name = "sayakpaul/vit-base-patch16-224-in21k-finetuned-lora-food101"
image_processor = AutoImageProcessor.from_pretrained(repo_name)
encoding = image_processor(image.convert("RGB"), return_tensors="pt")

preprocessor_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

In [6]:
from peft import LoraConfig

config = {
  "base_model_name_or_path": "google/vit-base-patch16-224-in21k",
  "bias": "none",
  "fan_in_fan_out": False,
  "inference_mode": True,
  "lora_alpha": 16,
  "lora_dropout": 0.1,
  "modules_to_save": [
    "classifier"
  ],
  "peft_type": "LORA",
  "r": 16,
  "target_modules": [
    "query",
    "value"
  ],
  "task_type": None
}
peft_config = LoraConfig(**config)
peft_config.base_model_name_or_path

'google/vit-base-patch16-224-in21k'

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    peft_config.base_model_name_or_path,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
# Load the LoRA model
inference_model = PeftModel.from_pretrained(model, repo_name, config=peft_config)
# inference_model.load_adapter(repo_name, adapter_name="adapter_a", config=peft_config)
inference_model

In [11]:
inference_model.to("cuda")
inference_model = inference_model.eval()

In [ ]:
encoding.to("cuda")

In [23]:
%%time
import torch
with torch.no_grad():
    for _ in range(100):
      outputs = inference_model(**encoding)

CPU times: user 1.79 s, sys: 12.3 ms, total: 1.8 s
Wall time: 1.83 s


In [24]:
%%time
inference_model.merge_and_unload()

CPU times: user 231 ms, sys: 5.78 ms, total: 237 ms
Wall time: 246 ms


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [32]:
%%time
import torch
with torch.no_grad():
    for _ in range(100):
      outputs = inference_model(**encoding)

CPU times: user 1.24 s, sys: 114 ms, total: 1.36 s
Wall time: 1.36 s
